이번 장에서는 고성능 함수형 프로그램을 작성할 때 사용할 수 있는 최적화 기법을 몇 가지 알아본다. 메모이제이션 알고리즘을 구현하는 방법에는 여러 가지가 있다. 또한 데코레이터를 작성하는 방법도 설명할 것이다. 더 중요한 것은 Callable 객체를 사용해 메모이제이션 한 결과를 캐시에 저장하는 방법을 보게될 것이라는 점이다.

### 메모이제이션과 캐싱

설계를 바꾸려면 약간의 생각이 핋요하다. 이에 대해 메모이제이션을 사용한 버전은 상당히 빠르지만, 많은 생각을 하여 설계를 바꿀 필요가 없다. 

### 메모이제이션 특화시키기

메모이제이션 핵심 아이디어는 @lru_cache 데코레이터에 넣을 수 있을 만큼 간단하다. 

여러 내부 캐시를 포함하는 Callable 객체를 만들 것이다. 

In [1]:
from functools import reduce
from operator import mul
prod = lambda x: reduce (mul, x)

다음은 prod 함수를 사용하는 캐시가 둘 있는 Callable 객체다.

In [7]:
from collections.abc import Callable
class Binomial(Callable):
    def __init__(self):
        self.fact_cache = {}
        self.bin_cache = {}
    def fact(self, n):
        if n not in self.fact_cache:
            self.fact_cache[n] = prod(range(1, n+1))
        return self.fact_cache[n]
    def __call__(self, n, m):
        if (n, m) not in self.bin_cache:
            self.bin_cache[n, m] = self.fact(n) // (self.fact(m)*self.fact(n-m))
        return self.bin_cache[n, m]

In [8]:
binom = Binomial()
binom(52, 5)

2598960

### 꼬리재귀 최적화

재귀를 설계하려면 다음 명령을 실행해야 한다.

In [10]:
def fact(n):
    if n == 0: return 1
    else: return n*fact(n-1)

In [11]:
def fact(n):
    if n ==0: return 1
    f = 1
    for i in range(2, n):
        f = f*1
    return f

그 어떤 최적화보다 먼저 확인해야 할 것은 해당 함수가 잘 작동하는지의 여부다. 이를 확인하기 위해 간단한 독테스트 문자열만으로 충분한 경우도 많다. 계승 함수에 다음과 같은 설명을 덧붙일 수 있을 것이다.

In [12]:
def fact(n):
    """Recursive Factorial
    >>> fact(0)
    1
    >>> fact(1)
    1
    >>> fact(7)
    5040
    """
    if n == 0: return 1
    else: return n*fact(n-1)

좀 더 복잡한 함수 조합을 사용한다면, 다음과 같은 명령을 실행해야 할 수도 있다.

In [14]:
test_example= """
>>> binom = Binomial()
>>> binom(52, 5)
2598960
"""

__test__ = {
    "test_example":test_example,
}

\__test__ 변수는 doctest.testmod() 함수가 사용하는 것이다. \__test__ 변수와 연관된 딕셔너리에 있는 모든 값도 독테스트 문자열처럼 검색 대상이 된다. 이를 통해 여러 함수를 합성한 특징을 테스트할 수 있다. 이를 통합테스트라고 부른다. 

일련의 테스트가 포함된 코드가 있다면 최적화 시 자신감을 가질 수 있고, 최적화의 올바름을 쉽게 확인할 수 있다. 다음은 최적화를 표현할 때 자주 쓰이는 유명한 인용문이다.

* 잘못된 프로그램을 더 나쁘게 만드는 것의 죄악이다.

여기서 중요한 것은 올바른 코드만을 최적화해야 한다는 것이다. 

### 메모리 최적화 

파이썬에서 사용할 수 있는 메모리 최적화 기법 중 하나는 반복 가능 객체를 사용하는 것이다. 반복 가능 객체는 실체화한 컬렉션의 특성을 상당수 포함하지만, 컬렉션이 사용하는 만큼 메모리를 쓸 필요는 없다. 반복 가능 객체에 대해 사용할 수 없는 연산이 극소수 있다. 예로 len()이 있다. 

### 정확도 최적화

계산의 정확도를 최적화할 필요가 있는 경우도 약간 존재한다. 정확도 최적화는 상당히 어려우며, 선택학 접근 방법의 정확도의 한계를 결정하려면 상당한 고등 수학이 필요할 수도 있다.

파이썬에서 할 수 잇는 재미있는 정확도 최적화로는 부동 소수점 수를 사용한 근사값 fractions.Faction 값으로 바꾸는 것이다. 일부 애플리케이션의 경우, 이렇게 하면 부동 소수점 수의 가수보다 더 많은 비트를 분모와 분자에 사용하기 때문에 훨씬 더 정확한 결과를 만들 수 있다.

decimal.Decimal 값을 정확한 계산이 필요한 경우에 사용할 수 있다는 점이 중요하다. 그러한 경우 float 값을 사용하는 것은 흔히 저지르는 실수다. float 값을 사용하면 입력의 10진 소수 값과 그 값을 2진수로 근사한 부동 소수점 수 사이의 차이로 인해 잡은 비트가 들어갈 수 있다. Decimal 값을 사용하면 이러한 매우 작업 차이가 발생하는 것을 막을 수 있다.

초월 함수를 사용하는 경우라면 이렇게 변경해도 그렇게 이익이 없을 것이다. 초월 함수는 정의에 의해 무리수로 이뤄져 있다.

### 고객의 요구에 맞춰 정확도를 감소시키기

